
# LCEL 완전 기초 실습 노트북 (LangChain 0.3.x)  
**Runnable/LCEL** 기초부터 NVIDIA API 연동까지 한 번에 익히는 주피터 노트북입니다.  
목표:
1) LCEL 핵심 컨셉 이해: Runnable, `|` 파이프, `invoke/batch/stream`  
2) 입력/출력 가공, 병렬 실행, 프롬프트/파서 사용법  
3) **NVIDIA API 모델**을 LCEL 노드로 감싸서 스트리밍/단발 호출하기  
4) 간단 체인: 입력 → 프롬프트 → NVIDIA LLM → 파서

> 본 노트북은 **LangChain 0.3.x** 버전에 맞는 임포트 경로를 사용합니다.



## 0. 환경 점검 & 필요한 패키지 설치
주피터 **현재 커널**에 바로 설치합니다. (커널/인터프리터 불일치 이슈 방지)


In [1]:

import sys, subprocess, importlib

def ensure(pkgs):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", *pkgs])

base_pkgs = [
    "langchain-core>=0.3.72",
    "langchain>=0.3.27",
]
ensure(base_pkgs)

print("Python:", sys.executable)
import langchain, langchain_core
print("langchain:", langchain.__version__)
print("langchain_core:", langchain_core.__version__)
print("langchain_core 위치:", langchain_core.__file__)


Python: C:\Users\jaeye\anaconda3\python.exe
langchain: 0.3.27
langchain_core: 0.3.74
langchain_core 위치: C:\Users\jaeye\anaconda3\Lib\site-packages\langchain_core\__init__.py



## 1. LCEL 최소 단위: Runnable + 파이프 + `invoke()`
- **RunnableLambda**: 파이썬 함수를 Runnable로 감싸서 LCEL 파이프라인에 연결 가능  
- `|` 연산자: **체인 연결**(앞 결과 → 뒤 입력)  
- `invoke(x)`: 단일 입력 실행


In [3]:

from langchain_core.runnables import RunnableLambda

def add_five(x): 
    return x + 5

def multiply_by_two(x): 
    return x * 2

add = RunnableLambda(add_five)
mul2 = RunnableLambda(multiply_by_two)

chain_basic = add | mul2  # (x + 5) -> * 2

print("chain_basic.invoke(10) =", chain_basic.invoke(10))  # 30 기대


chain_basic.invoke(10) = 30



## 2. 입력/출력 가공: `RunnableMap`, `RunnablePassthrough`
- **RunnableMap**: 입력 하나로 **여러 키**를 생성 (전처리/특징추출에 유용)  
- **RunnablePassthrough**: 원본 입력을 그대로 전달 (스킵 연결용)


In [6]:

from langchain_core.runnables import RunnableMap, RunnablePassthrough, RunnableLambda

prepare = RunnableMap({
    "orig": RunnablePassthrough(),
    "plus5": RunnableLambda(lambda x: x + 5),
})

use_plus5 = RunnableLambda(lambda d: d["plus5"] * 2)
#use_plus5 = RunnableLambda(lambda d: d["orig"] * 2)

chain_map = prepare | use_plus5
print("chain_map.invoke(10) =", chain_map.invoke(10))  # 30


chain_map.invoke(10) = 30



## 3. 병렬 실행: `RunnableParallel`
동일 입력을 **여러 갈래로 병렬 처리** 후 dict로 결과를 받습니다.


In [10]:

from langchain_core.runnables import RunnableParallel, RunnableLambda

parallel = RunnableParallel({
    "square": RunnableLambda(lambda x: x * x),
    "cube":   RunnableLambda(lambda x: x * x * x),
    "plus5":  RunnableLambda(lambda x: x + 5),
})

print("parallel.invoke(3) =", parallel.invoke(3))


parallel.invoke(3) = {'square': 9, 'cube': 27, 'plus5': 8}



## 4. 배치/스트림 실행: `.batch()`, `.stream()`
- **batch**: 리스트 입력 일괄 처리 → 결과 리스트  
- **stream**: 제너레이터 토큰 스트리밍 (LLM 토큰/프로그레스 등)


In [14]:

# 4-1. batch
print("chain_basic.batch([1,2,3,10]) =", chain_basic.batch([1, 2, 3, 10]))

# 4-2. stream: 간단한 글자 스트리머 예시
def stream_letters(text):
    for ch in text:
        yield ch

stream_node = RunnableLambda(stream_letters)

print("stream letters:", end=" ")
for t in stream_node.stream("HELLO"):
    print(t, end=" ")
    

chain_basic.batch([1,2,3,10]) = [12, 14, 16, 30]
stream letters: H E L L O 


## 5. 프롬프트 + 출력 파서: `ChatPromptTemplate`, `StrOutputParser`
- **ChatPromptTemplate**: 시스템/유저 등 역할 기반 프롬프트 구성  
- **StrOutputParser**: 문자열로 결과 파싱 (간단 케이스)
> 아직 LLM은 연결하지 않고, 포맷팅까지만 확인합니다.


In [15]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 한국어로 간결하게 답하는 도우미야."),
    ("human",  "다음 문장을 요약해줘:\n\n{input}")
])

formatted = prompt.invoke({"input": "LangChain은 파이프라이닝이 쉬운 LCEL을 제공합니다."})
print("포맷 결과 메시지들:", formatted.to_messages())


포맷 결과 메시지들: [SystemMessage(content='너는 한국어로 간결하게 답하는 도우미야.', additional_kwargs={}, response_metadata={}), HumanMessage(content='다음 문장을 요약해줘:\n\nLangChain은 파이프라이닝이 쉬운 LCEL을 제공합니다.', additional_kwargs={}, response_metadata={})]



## 6. NVIDIA API를 LCEL 노드로 감싸기 (스트리밍 포함)
- 주어진 **NVIDIA Chat Completions** 엔드포인트 사용  
- **SSE(Stream)**를 파싱하여 토큰 단위로 출력  
- LCEL과 연결하기 위해 **RunnableLambda**로 감쌉니다.

> **API 키 필요**: `nvapi-`로 시작하는 **NVIDIA_API_KEY**를 환경변수로 넣거나, 노트북에서 입력받습니다.
>
> NVIDIA API는 OpenAI 호환 Chat Completions 엔드포인트를 제공합니다.
아래 코드는 스트리밍으로 토큰을 받아 출력하는 예시입니다.
모델: mistralai/mixtral-8x7b-instruct-v0.1
API 키는 다음 링크에서 받을 수 있습니다. [https://build.nvidia.com/]


In [37]:

import os, json, requests
from getpass import getpass
from langchain_core.runnables import RunnableLambda

#os.environ["NVIDIA_API_KEY"] = "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"

# 🔑 키 설정
if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    os.environ["NVIDIA_API_KEY"] = getpass("NVIDIA_API_KEY를 입력하세요 (nvapi-...): ")

INVOKE_URL = "https://integrate.api.nvidia.com/v1/chat/completions"
HEADERS = {
    "accept": "text/event-stream",
    "content-type": "application/json",
    "Authorization": f"Bearer {os.environ['NVIDIA_API_KEY']}",
}

def _extract_stream_token(entry: bytes) -> str:
    if not entry:
        return ""
    try:
        if entry.startswith(b"data: "):
            payload = json.loads(entry[6:].decode("utf-8"))
            return payload.get("choices", [{}])[0].get("delta", {}).get("content") or ""
    except Exception:
        return ""
    return ""

def nv_chat_stream(messages,
                   model="mistralai/mixtral-8x7b-instruct-v0.1",
                   temperature=0.5,
                   top_p=1,
                   max_tokens=256):
    payload = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "top_p": top_p,
        "max_tokens": max_tokens,
        "stream": True
    }
    with requests.post(INVOKE_URL, headers=HEADERS, json=payload, stream=True) as r:
        r.raise_for_status()
        for line in r.iter_lines():
            tok = _extract_stream_token(line)
            if tok:
                yield tok

def nv_chat_once(messages,
                 model="mistralai/mixtral-8x7b-instruct-v0.1",
                 temperature=0.5,
                 top_p=1,
                 max_tokens=256) -> str:
    return "".join(nv_chat_stream(messages, model, temperature, top_p, max_tokens))

def _nv_node_fn(user_input: str) -> str:
    msgs = [{"role": "user", "content": user_input}]
    return nv_chat_once(msgs)

nv_node = RunnableLambda(_nv_node_fn)

print("NVIDIA 노드 단발 호출 테스트:")
print(nv_node.invoke("안녕? 한국어로 짧게 인사해줘."))

print("NVIDIA 스트리밍 테스트:")
for tok in nv_node.stream("너는 한국어를 쓰는 모델이야. 오늘 기분이 어때?"):
    print(tok, end="")
print()


NVIDIA 노드 단발 호출 테스트:
안녕하세요!  Korean greeting.  How can I help you today?
NVIDIA 스트리밍 테스트:
안녕하세요! 현재 모델은 기분이 VERY GOOD 상태입니다. 매일 새로운 데이터와 경험을 통해 더 뛰어난 서비스를 제공하기 위해 노력하고 있습니다. 언제나 질문이 있으시면  warmly welcome  your inquiries. 감사합니다.



## 7. 조합: 입력 → 프롬프트 → NVIDIA LLM → 문자열 파서 (LCEL 체인)
- 프롬프트 출력(Messages)을 **단일 user 메시지**로 합쳐 NVIDIA 포맷에 맞춥니다.  
- LCEL 파이프: `RunnablePassthrough`로 입력을 dict화 → `ChatPromptTemplate` → 변환 → NVIDIA 호출 → `StrOutputParser`


In [36]:

from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

summ_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 전문 요약가야. 한국어로 핵심만, 3줄 이내로 요약한다."),
    ("human",  "다음 내용을 요약해줘:\n\n{input}")
])

def to_nvidia_messages(template_out) -> list:
    msgs = template_out.to_messages()
    text = "\n".join(m.content for m in msgs)
    return [{"role": "user", "content": text}]

def call_nvidia_with_messages(messages: list) -> str:
    return nv_chat_once(messages)

nv_messages_node = RunnableLambda(to_nvidia_messages)
nv_call_node = RunnableLambda(call_nvidia_with_messages)
to_str = StrOutputParser()

ragless_chain = (
    {"input": RunnablePassthrough()}
    | summ_prompt
    | nv_messages_node
    | nv_call_node
    | to_str
)

print(ragless_chain.invoke("미국의 트럼프 대통령에 대해 소개해줘"))


- Donald Trump는 2017년 1월 1일 ~ 2021년 1월 20일까지 미국 대통령이었습니다.
- 트럼프는 비즈니스  Tycoon으로서 유명했으며, 미국 정치에 처음 입문한 것은 2015년입니다.
- 그는 북한 핵 위협, 이민 문제 등 다양한 국제적, 국내적 문제에 대응하며 활약했습니다.



## 마무리
- 여기까지로 **LCEL 핵심 흐름**과 **NVIDIA API 연동**을 익혔습니다.